In [1]:
# sorry for the variable names
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

sia = SentimentIntensityAnalyzer()

# load mo yung csv na dinownload mo sa google forms, ibahin mo nalang yung path pag di gumana sayo
df1 = pd.read_csv("ds/Sentiment Analysis on DLSU-D Tuition Fee Increase.csv")

# ignore the first two columns ("Timestamp" and "Name")
question_columns = list(df1.columns.values[2:])

# the questions used in the survey
print("Questions")
question_num = 1
for question in question_columns:
    print(f"{question_num}. {question}")
    question_num += 1
print("\n")

# replace the column names with a shorter one for convenience
question_num = 1
for column in df1:
    if column == "Timestamp" or column == "Name":
        pass
    else:
        df1.rename(columns={column:f"Q{question_num}"}, inplace=True)
        question_num += 1
        
question_columns_processed = list(df1.columns.values[2:])

def process_dataframe(df):
    """Gets the polarity scores and other data about an answer."""
    
    # clean data
    # drop NA values
    df.dropna(inplace=True)
    
    # drop spaces
    blanks = []
    for index, answer in df.itertuples():
          if type(answer) == str:
                if answer.isspace():
                    blanks.append(index)

    df.drop(blanks, inplace=True)

    # df.iloc[:,0] means the first column, regardless of what dataframe you're trying to process
    df["polarity_scores"] = df.iloc[:,0].apply(lambda review: sia.polarity_scores(review))
    df["compound"] = df["polarity_scores"].apply(lambda score_dict: score_dict["compound"])
    df["compound_score"] = df["compound"].apply(lambda score: "pos" if score >= 0 else "neg")
    
    # drop neutral compound scores
    blanks = []
    for index, answer, polarity, compound, compound_score in df.itertuples():
          if float(compound) == 0:
                blanks.append(index)

    df.drop(blanks, inplace=True)
    
    # hula lang tong part na to
    df["label"] = df["polarity_scores"].apply(lambda score_dict: "pos" if score_dict["pos"] >= score_dict["neg"] else "neg")
    
    # most likely wrong
    # df["label"] = df["polarity_scores"].apply(lambda score_dict: "pos" if score_dict["pos"] or score_dict["neu"] > score_dict["neg"] else "neg")

def process_dataframe_metrics(df):
    """Does the matrix thing."""
    # doon sa example ni sir, yung dataset na ginamit nya may column na nakapangalan na
    # "label". kapag dinownload mo yung csv doon sa survey natin, walang column na ganon
    # yung pangalan. so baka literal na mano-mano natin lalagyan yung bawat isang answer
    # doon sa dataset, kung ang sagot ba nila ay positive o negative. di ko rin alam eh.
    # in the meantime, walang sense tong function na to, kaya wag muna galawin.
    print("Accuracy Score:", accuracy_score(df["label"], df["compound_score"]))
    print("Classification Report:\n", classification_report(df["label"], df["compound_score"]))
    print("Confusion Matrix:\n", confusion_matrix(df["label"], df["compound_score"]))

    # garbage
    """
    print("Accuracy Score:", accuracy_score(df["compound_score"], df["compound_score"]))
    print("Classification Report:\n", classification_report(df["compound_score"], df["compound_score"]))
    print("Confusion Matrix:\n", confusion_matrix(df["compound_score"], df["compound_score"]))
    """

def generate_question_df(df):
    """Gets all answers for each question, and then processes each answer."""
    question_df = []
    question_num = 1
    for column in question_columns_processed:
        question_df.append(pd.DataFrame({f"Q{question_num}": list(df.loc[:, f"Q{question_num}"])}))
        question_num += 1 
    for df in question_df:
        process_dataframe(df)
    return question_df      

# use 0 for the first question, 9 for last
EXAMPLE = 1

# get list of questions with their corresponding responses
question_df_list = generate_question_df(df1)

# process 1 dataframe related to that question to get the metrics
process_dataframe_metrics(question_df_list[EXAMPLE])

# print the question being asked
print("\nQuestion:", question_columns[EXAMPLE])

# print all responses to that question and its corresponding data
question_df_list[EXAMPLE].head(99999)

Questions
1. Are you satisfied with your course? Why do you feel like it?
2. Do you think the course that you chose is worth it? Why?
3. Is the tuition fee increase justifiable? Give insight on how it affects you.
4. Do you think that they do this to catch up with other institutions? If so, why do you think that is?
5. Are you satisfied with the materials and equipment that the university provides you? Elaborate and provide feedback, if able.
6. Is the tuition fee increase manageable? How do you think to manage it?
7. How will you plan for future tuition fee increases?
8. What additional expenses should you expect to pay for besides the tuition fee?
9. What external factors do you think affects how you manage the tuition fee?
10. How should students cope with future tuition fee increases?


Accuracy Score: 0.9666666666666667
Classification Report:
               precision    recall  f1-score   support

         neg       1.00      0.80      0.89         5
         pos       0.96      1

,Q2,polarity_scores,compound,compound_score,label
0,"Yes, because it would enhance and let me learn...","{'neg': 0.078, 'neu': 0.706, 'pos': 0.216, 'co...",0.5423,pos,pos
1,Yes because Comp Sci related jobs is highly fa...,"{'neg': 0.0, 'neu': 0.621, 'pos': 0.379, 'comp...",0.7264,pos,pos
2,Yes. I think the course I chose is worth it si...,"{'neg': 0.068, 'neu': 0.724, 'pos': 0.208, 'co...",0.4767,pos,pos
3,Not so much if I am being honest. Like I have ...,"{'neg': 0.14, 'neu': 0.726, 'pos': 0.133, 'com...",0.1536,pos,neg
4,"Yes, since I know that I will be happy once I ...","{'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'comp...",0.9153,pos,pos
5,The course that i chose is worth it as i enjoy...,"{'neg': 0.071, 'neu': 0.736, 'pos': 0.193, 'co...",0.5423,pos,pos
6,"Yes, it is very worth it for me, because I lik...","{'neg': 0.0, 'neu': 0.52, 'pos': 0.48, 'compou...",0.7501,pos,pos
7,No. The tuition fee is too expensive for the s...,"{'neg': 0.128, 'neu': 0.872, 'pos': 0.0, 'comp...",-0.2960,neg,neg
8,"Yes, because it gives me plenty of opportuniti...","{'neg': 0.0, 'neu': 0.629, 'pos': 0.371, 'comp...",0.6486,pos,pos
9,Yes. I learn new things from it and I can see ...,"{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'comp...",0.4019,pos,pos


# mali tong part na to ↓↓↓

In [2]:
import nltk
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

sia = SentimentIntensityAnalyzer()

df2 = pd.read_csv("ds/Sentiment Analysis on DLSU-D Tuition Fee Increase.csv")

question_columns = list(df.columns.values[2:])

# the questions used in the survey
print("Questions")
question_num = 1
for question in question_columns:
    print(f"{question_num}. {question}")
    question_num += 1
print("\n")

# replace column names with a shorter name for convenience
question_num = 1
for column in df2:
    if column == "Timestamp" or column == "Name":
        pass
    else:
        df.rename(columns={column:f"Q{question_num}"}, inplace=True)
        question_num += 1
        
question_columns_processed = list(df2.columns.values[2:])

print("Accuracy Score:", accuracy_score(question_columns_processed))
# print("Classification Report:\n", classification_report(df["label"], df["compound_score"]))
# print("Confusion Matrix:\n", confusion_matrix(df["label"], df["compound_score"]))

NameError: name 'df' is not defined

In [ ]:
answers = [
    "Neutral. I believe that we have both pros and cons on choosing the course.",
    "I feel that it is lacking, with the increase of the tuition fee and comparing it to the facilities offered to us. It is not fair that we have such high tuition fee yet have low-quality facilities and computers. I am not satisfied, I feel that it is not good.",
    "Yes, I am satisfied. It is very fun learning new and different stuff relating to computers.",
    "Yes! Because I love what I do.",
    "No. Because I do not have any access to laboratory room and equipment.",
    "This product sucks. I want my fucking money back!"
]

for answer in answers:
    print(answer[:20], "...", sia.polarity_scores(answer))